In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import json
import sys
import os
from pathlib import Path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.refinement import knowledge

In [2]:
%run utils/ranks.py
%run utils/attention_graph.py
%run utils/mlflow_query.py
%run utils/percentiles.py
%run utils/loading.py
%run utils/comparison.py
%run utils/refinement.py

In [3]:
mlflow_helper = MlflowHelper(pkl_file=Path("mlflow_run_df.pkl"))
#mlflow_helper.query_all_runs(pkl_file=Path("mlflow_run_df.pkl"))

Initialized with 1758 MLFlow runs from pkl


## Get a GRAM run-id to compare

In [4]:
relevant_gram_ref_df = mlflow_helper.mimic_run_df(include_noise=False, include_refinements=True)

relevant_gram_ref_df = relevant_gram_ref_df[
        relevant_gram_ref_df["data_tags_refinement_type"].fillna("").astype(str).apply(len) > 0
].copy()
relevant_gram_ref_df['refinement_run'] = relevant_gram_ref_df["data_tags_refinement_type"].apply(lambda x: x.split("_")[0])
relevant_gram_ref_df['refinement_type'] = relevant_gram_ref_df["data_tags_refinement_type"].apply(lambda x: "_".join(x.split("_")[1:]))

In [5]:
relevant_gram_ref_df

,info_run_id,data_metrics_categorical_accuracy,data_metrics_epoch,data_metrics_loss,data_metrics_num_connections,data_metrics_num_features,data_metrics_num_hidden_features,data_metrics_test_size,data_metrics_top_10_categorical_accuracy,data_metrics_top_10_categorical_accuracy_cp0,...,data_metrics_val_recall,data_params_HuaweiPreprocessorConfigdrain_log_depth,data_params_HuaweiPreprocessorConfigdrain_log_st,data_tags_noise_type,data_params_ExperimentConfigattention_weight_reference_threshold,data_params_HuaweiPreprocessorConfigdatetime_column_name,data_params_HuaweiPreprocessorConfigmax_sequence_length,data_params_SequenceConfigremove_empty_v_vecs,refinement_run,refinement_type
64,b89719d074ce41388a563cf603c0a7e8,0.076112,47.0,0.234164,15622.0,4879.0,1157.0,20.0,0.507374,0.167109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1633177885.0532389,refinement_0
65,525e13d70ba74b2eac5209e2ca995d7d,0.075883,47.0,0.233998,15648.0,4879.0,1158.0,20.0,0.509116,0.173909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1633177885.0532389,original
66,0d8104b7a41546e3adb5bde8785b9154,0.075278,33.0,0.235550,4879.0,4879.0,1.0,20.0,0.500248,0.149773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1633177885.0532389,reference
67,af1a2b6c129044dd85b42a4393d6b9d1,0.076433,55.0,0.233110,15620.0,4879.0,1157.0,20.0,0.510840,0.179898,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1633175807.786,refinement_0
68,b84c2ae0e21a4acc8ed06ebbc8ce99bb,0.076773,51.0,0.233333,15648.0,4879.0,1158.0,20.0,0.510932,0.180934,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1633175807.786,original
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,2a61eda2013043a887d991bab1f90e81,0.076984,52.0,0.232845,26578.0,4879.0,1953.0,20.0,0.515564,0.187869,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1630598173.3106875,refinement_2
534,de75ae3273b14b4fb1174426d7722f6f,0.076892,52.0,0.232932,26588.0,4879.0,1953.0,20.0,0.517251,0.187689,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1630598173.3106875,refinement_1
535,b0e1c77d156242119ae120d7dc16ad53,0.076699,54.0,0.232590,26598.0,4879.0,1953.0,20.0,0.518553,0.194443,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1630598173.3106875,refinement_0
536,afe9fb9b296f41759feca9bfeecdf862,0.076965,44.0,0.234473,26608.0,4879.0,1955.0,20.0,0.509254,0.164228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1630598173.3106875,original


In [7]:
gram_example_runs = {
    'edges_added': {
        'gram': relevant_gram_ref_df[
            (relevant_gram_ref_df['data_params_RefinementConfigoriginal_file_knowledge'] == 'data/gram_original_file_knowledge.json') &
            (relevant_gram_ref_df['data_params_RefinementConfigedges_to_add'].fillna('0.0') == '0.1') 
        ]['refinement_run'].iloc[0],
    },
    'edges_removed': {
        'gram': relevant_gram_ref_df[
            (relevant_gram_ref_df['data_params_RefinementConfigoriginal_file_knowledge'] == 'data/gram_original_file_knowledge.json') &
            (relevant_gram_ref_df['data_params_RefinementConfigedges_to_add'].fillna('0.0') == '0.0') 
        ]['refinement_run'].iloc[0],
    },
}

gram_example_runs

{'edges_added': {'gram': '1631557891.6481'},
 'edges_removed': {'gram': '1631270105.486519'}}

In [8]:
original_run_id=relevant_gram_ref_df[
    (relevant_gram_ref_df['refinement_run'] == gram_example_runs['edges_removed']['gram']) &
    (relevant_gram_ref_df['refinement_type'] == "original")
]["info_run_id"].iloc[0]

reference_run_id=relevant_gram_ref_df[
    (relevant_gram_ref_df['refinement_run'] == gram_example_runs['edges_removed']['gram']) &
    (relevant_gram_ref_df['refinement_type'] == "reference")
]["info_run_id"].iloc[0]

## Get a Simple run-id to compare

In [33]:
relevant_simple_run_df = mlflow_helper.mimic_run_df(include_noise=False, include_refinements=False)

In [34]:
mimic_simple_run = relevant_simple_run_df[
        (relevant_simple_run_df['data_tags_model_type'] == 'simple') &
        (relevant_simple_run_df['data_params_ModelConfigbase_hidden_embeddings_trainable'] == 'True')
]

In [ ]:
print("rows: ", mimic_simple_run.shape[0],"columns: ", mimic_simple_run.shape[1])
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
mimic_simple_run

In [15]:
mimic_simple_run_id = mimic_simple_run.iloc[0].get('info_run_id')

print('Simple', mimic_simple_run_id)

Simple 18d4e833923f418ba10441feb507ccb6


## Find outlier from prediction results

In [ ]:
suffix_1 = '_simple_false' 
suffix_2='_gram_false'
comparison = Comparison(
    run_id_1=mimic_simple_run_id, 
    suffix_1=suffix_1, 
    run_id_2=mimic_simple_run_id, 
    suffix_2=suffix_2, 
    local_mlflow_dir=mlflow_helper.local_mlflow_dir,
    num_percentiles=10)
plot_rank_comparison(comparison, color="avg_input_frequencies_percentile")
plot_outlier_distances(comparison)
analyse_best_worst_sequences(comparison, num_best_sequences=1, num_worst_sequences=1, descriptions=load_icd9_text())

In [ ]:
suffix_1 = '_simple_false' 
suffix_2='_gram_false'
comparison = Comparison(
    run_id_1=mimic_simple_run_id, 
    suffix_1=suffix_1, 
    run_id_2=original_run_id, 
    suffix_2=suffix_2, 
    local_mlflow_dir=mlflow_helper.local_mlflow_dir,
    num_percentiles=10)
plot_rank_comparison(comparison, color="avg_input_frequencies_percentile")
plot_outlier_distances(comparison)
analyse_best_worst_sequences(comparison, num_best_sequences=1, num_worst_sequences=1, descriptions=load_icd9_text())

In [ ]:
suffix_1 = '_gram_ref' 
suffix_2='_gram_original'
comparison = Comparison(
    run_id_1=reference_run_id, 
    suffix_1=suffix_1, 
    run_id_2=original_run_id, 
    suffix_2=suffix_2, 
    local_mlflow_dir=mlflow_helper.local_mlflow_dir,
    num_percentiles=10)
plot_rank_comparison(comparison, color="avg_input_frequencies_percentile")
plot_outlier_distances(comparison)
analyse_best_worst_sequences(comparison, num_best_sequences=1, num_worst_sequences=1, descriptions=load_icd9_text())

## Find removed edges according to attention weight 

In [37]:
class RefinementConfig:
    min_edge_weight: float = 0.5
    max_train_examples: int = 500
    refinement_metric: str = "mean_outlier_score"
    refinement_metric_maxrank: int = 100
    max_edges_to_remove: int = 1000
    max_refinement_metric: int = -1
    mlflow_dir: str = "../mlruns/1/"

In [40]:
def plot_for_removed_edges(original_run_id, reference_run_id, local_mlflow_dir, use_node_mapping=False):
    original_attention = load_attention_weights(original_run_id, local_mlflow_dir)
    frequencies = load_input_frequency_dict(original_run_id, local_mlflow_dir)

    config = RefinementConfig()
    config.min_edge_weight = 0.5
    config.max_train_examples = 500
    config.max_refinement_metric = 0
    edge_comparison_df, refined_knowledge = knowledge.KnowledgeProcessor(config).load_refined_knowledge(refinement_run_id=original_run_id, reference_run_id=reference_run_id)
    
    print("==============here is refinement_metric in column 3 of edge_comparison_df========")
    print(edge_comparison_df)
          
    print("==============here is refined_knowledge set========")
    print(refined_knowledge)
    
    feature_node_mapping = convert_to_node_mapping(
        [x for x in original_attention], use_node_mapping
    )
    colored_connections = calculate_colored_connections(
        reference_connections=set(
            [(c,p) for c,ps in refined_knowledge.items() for p in ps]
        ),
        attention_weights=original_attention,
        feature_node_mapping=feature_node_mapping,
    )
    
    print("Removed", len(colored_connections), "edges")
    
    node_mapping = _create_graph_visualization(
        attention_weights=original_attention, 
        threshold=0.25, 
        run_name="refinement_edges_removed", 
        node_mapping=feature_node_mapping,
        colored_connections=colored_connections)
    
    return colored_connections, node_mapping, frequencies

def plot_for_added_edges(original_run_id, reference_run_id, local_mlflow_dir, use_node_mapping=False):
    original_attention = load_attention_weights(original_run_id, local_mlflow_dir)
    reference_attention = load_attention_weights(reference_run_id, local_mlflow_dir)
    frequencies = load_input_frequency_dict(original_run_id, local_mlflow_dir)

    config = RefinementConfig()
    config.min_edge_weight = 0.5
    config.max_train_examples = 50
    config.max_edges_to_remove = 1000
    config.max_refinement_metric = 2
    refined_knowledge = knowledge.KnowledgeProcessor(config).load_refined_knowledge(refinement_run_id=original_run_id, reference_run_id=reference_run_id)
    
    
    refined_attention = {c:{} for c in refined_knowledge}
    for child in original_attention:
        for parent in original_attention[child]:
            if parent in refined_knowledge.get(child, {}):
                refined_attention[child][parent] = original_attention[child][parent]

    feature_node_mapping = convert_to_node_mapping(
        [x for x in original_attention], use_node_mapping
    )
    colored_connections = calculate_colored_connections(
        reference_connections=set(
            [(c,p) for c,ps in reference_attention.items() for p in ps]
        ),
        attention_weights=refined_attention,
        feature_node_mapping=feature_node_mapping,
    )
    print("Added", len(colored_connections), "edges")
    node_mapping = _create_graph_visualization(
        attention_weights=refined_attention, 
        threshold=0.75, 
        run_name="refinement_edges_added", 
        node_mapping=feature_node_mapping,
        colored_connections=colored_connections)
    return node_mapping, node_mapping, frequencies

In [41]:
colored_connections, node_mapping, frequencies = plot_for_removed_edges(
    original_run_id,
    reference_run_id, 
    local_mlflow_dir=mlflow_helper.local_mlflow_dir, 
    use_node_mapping=False,
)

100%|████████████████████████████████████| 21729/21729 [02:01<00:00, 179.31it/s]


==============here is refinement_metric in column 3 of edge_comparison_df========
       child     parent  refinement_metric
1499  674.54    630-679          -6.363961
1831  611.79        611          -5.112926
339   873.54    800-999          -4.949747
1605  E965.0       E965          -4.949747
1809  802.24        802          -4.949747
448   802.25        802          -4.949747
1509  719.02    710-739          -4.949747
1567  852.01    800-999          -4.714045
1049  349.39    340-349          -4.384062
1867  V85.32        V85          -4.384062
1220  402.90      402.9          -3.623922
1216  719.42    710-739          -3.535534
1549  851.80    800-999          -3.394113
1456  453.85        453          -3.381815
1043  747.49    740-759          -3.181981
110   E938.4       E938          -3.064129
1638   289.3        289          -2.755278
568   372.03    360-379          -2.730895
744    565.0    560-569          -2.607456
1338   333.4    320-389          -2.592725
545   765.26   

## Compare the two sets of removed edges